In [ ]:
import sys
from _util import * 

In [ ]:
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup
import math
import re
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver

In [ ]:
import numpy as np
from concurrent.futures import ThreadPoolExecutor
import os
from datetime import datetime
from _helpers_parsing import driver_setup
from _util import *

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
esco_jobs = load_jobs()

In [ ]:
def get_eures_overview(esco_codes,driver,filename,patience):
    codedict =[]
    for code in tqdm(esco_codes):
        uri = code["uri"].split("/")[-1]
        driver.get(f"https://ec.europa.eu/eures/portal/jv-se/search?page=1&resultsPerPage=50&orderBy=MOST_RECENT&availableLanguages=de&escoOccupation={uri}&lang=de")
        try:
            WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.ID, "jv-no-result")))
            continue
        except:
            pass
        try:
            WebDriverWait(driver, patience).until(EC.presence_of_element_located((By.CLASS_NAME, "ecl-u-type-heading-2")))
            time.sleep(1)
            soup = BeautifulSoup(driver.page_source)
            results = (soup.find(class_ = "ecl-u-type-heading-2").text.split(" ")[2])
            results = int("".join([c for c in results if c.isdigit()]))
            #always 50 results per page, therefore /50
            pages = math.ceil(results/50)
            # set pages to 6 to get 6*50 --> top 300 latest results
            if pages > 6:
                pages = 6
        except:
            #print(f"skipped {code['jobtitle']}")
            codedict.append({"searched_esco_job":code["jobtitle"]})
            continue
        for i in (range(pages)):
            driver.get((f"https://ec.europa.eu/eures/portal/jv-se/search?page={i+1}&resultsPerPage=50&orderBy=BEST_MATCH&availableLanguages=de&escoOccupation={uri}&lang=de"))
            try:
                WebDriverWait(driver, patience).until(EC.presence_of_element_located((By.CLASS_NAME, "jv-result-job-category")))
            except:
                continue
            time.sleep(1)
            soup = BeautifulSoup(driver.page_source)
            hits = soup.find_all("article")
            for hit in hits:
                url = "https://ec.europa.eu"+hit.find(href=True)["href"]
                title = hit.find(href=True).text
                publication_date = hit.find("em").text.replace(" ","").replace("/",".")
                esco_jobs= []
                for x in (hit.find_all("span",{"class":"jv-result-job-category"})):
                    esco_jobs.append(x.text.strip(", "))
                job = {"searched_esco_job":code["jobtitle"], "title":title,"url":url,"esco_jobs":esco_jobs,"publication_date":publication_date}
                codedict.append(job)
        with open(filename, 'w',encoding= "utf-8") as fp:
            json.dump(codedict, fp, indent = 2, ensure_ascii=False)
    return codedict

In [ ]:
# multithreading the parsing, to speed up the process
def multithread_eures_overview(esco_joblist, headless=True,patience=10):
    
    drivers = [driver_setup(headless) for _ in range(4)]
    currently = "".join([c for c in str(datetime.now()).split('.')[0] if c.isdigit()])
    filenames = [f"eures_overview/{currently}_eres_overview_{i}.json" for i in range(1,5)]
    patience = [patience for _ in range(4)]

    chunks = np.array_split(esco_joblist, 4)
    with ThreadPoolExecutor(max_workers=4) as executor:  
        bucket = executor.map(get_eures_overview,chunks,drivers,filenames,patience)
        results = [item for block in bucket for item in block]
    write_json("eures_overview/eures_overview_total.json",results)
    [driver.quit() for driver in drivers]
    return results

In [ ]:
x = multithread_eures_overview(esco_jobs, headless=False,patience=10)

In [ ]:
import pandas as pd

In [ ]:
import os
parsedads = pd.DataFrame(flatten_list([load_json("eures_overview/"+str(x)) for x in os.listdir('eures_overview')]))

In [ ]:
len(parsedads.drop_duplicates("url"))

In [ ]:
print(len(parsedads))
parsedads = parsedads.drop_duplicates("url")
parsedads = parsedads.dropna()
esco_jobs_parsed = flatten_list([x["esco_jobs"] for x in tqdm(parsedads.to_dict("records"))])

In [ ]:
len(set(esco_jobs_parsed))

In [ ]:
parsedads["no_escojobs"] = parsedads["esco_jobs"].apply(len)
esco_jobs_parsed = flatten_list([x["esco_jobs"] for x in tqdm(parsedads[parsedads["no_escojobs"]==1].to_dict("records"))])

# Output

In [ ]:
import pandas as pd
from _util import *

In [ ]:
results = pd.DataFrame(load_json("../00_data/EURES/eures_overview_total.json"))

In [ ]:
results.head(1)

In [ ]:
print(f"A total of {len(results)} links to job advertisements were parsed.")
results = results.drop_duplicates(["title", "url"])
print(f"Without duplicates there are {len(results)} job ad links available.")